In [3]:
## template
import math
from typing import List, Optional, Union

import torch

def num_swapped_pairs(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> int:
    pass


def compute_gain(y_value: float, gain_scheme: str) -> float:
    pass


def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    pass


def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    pass


def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
    pass


def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    pass


def p_found(ys_true: torch.Tensor, ys_pred: torch.Tensor, p_break: float = 0.15 ) -> float:
    pass


def average_precision(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    pass

# all funs

In [5]:
import math
from typing import List, Optional, Union

import torch

def num_swapped_pairs(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> int:
    
    n_pairs = 0
    ys_true_sorted = torch.sort(ys_true, descending=True)
    
    for i in range(len(ys_true_sorted[0]) - 1):
        for j in range(i, len(ys_true_sorted[0])):
            if (ys_pred[ys_true_sorted[1][i]] < ys_pred[ys_true_sorted[1][j]]) & \
                (ys_true_sorted[0][i] != ys_true_sorted[0][j]):
                n_pairs += 1
    
    return n_pairs

def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'exp2':
        return 2 ** y_value - 1.
    else:
        return y_value + 0.

def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    log2_list = [math.log2(x) for x in range(2, len(ys_pred) + 2)]
    
    dcg_val = 0.
    for i in range(len(log2_list)):
        dcg_val += compute_gain(ys_true[ys_pred_sorted[1]][i].item(), \
                                gain_scheme=gain_scheme) / log2_list[i]
    
    return dcg_val

def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    ndcg_val = dcg(ys_true=ys_true, ys_pred=ys_pred, gain_scheme=gain_scheme) / \
        dcg(ys_true=ys_true, ys_pred=ys_true, gain_scheme=gain_scheme)
    return ndcg_val

def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
    k_max = k
    k = min(k, len(ys_true))
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]]

    ys_true_k = ys_true_sorted[:k]
    ys_pred_k = ys_pred_sorted[0][:k]
    
    if sum(ys_true).item() == 0:
        prec_k_val = -1
    else:

        prec_k_val = sum(ys_true_k).item() / min(sum(ys_true).item(), k)
    
    return prec_k_val

def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]]
    
    if sum(ys_true_sorted.numpy() == 1) == 0:
        return 0
    else:
        ys_true_idx = ((ys_true_sorted == 1).nonzero(as_tuple=True)[0])
        return 1 / (ys_true_idx.item() + 1)

def p_found(ys_true: torch.Tensor, ys_pred: torch.Tensor, p_break: float = 0.15) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]] # pRel
    
    pLook = torch.zeros(len(ys_true))
    pLook[0] = 1
    
    for i in range(1, len(ys_true)):
        pLook[i] = pLook[i - 1] * (1 - ys_true_sorted[i - 1]) * (1 - p_break)
    
    pFound = sum(pLook * ys_true_sorted).item()
    
    return pFound

def average_precision(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    if sum(ys_true) == 0:
        return -1
    else:
        ys_pred_sorted = torch.sort(ys_pred, descending=True)
        ys_true_sorted = ys_true[ys_pred_sorted[1]]
        ys_true_sum = sum(ys_true).item()
        
        avg_prec = 0
        recall_k_prev = 0
        for i in range(len(ys_true)):
            recall_k = sum(ys_true_sorted[:(i + 1)]).item() / ys_true_sum
            prec_k = sum(ys_true_sorted[:(i + 1)]).item() / (i + 1)
            avg_prec_i = (recall_k - recall_k_prev) * prec_k
            recall_k_prev = recall_k
            
            avg_prec += avg_prec_i
            
        return avg_prec

# num_swapped_pairs

In [62]:
def num_swapped_pairs(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> int:
    
    n_pairs = 0
    ys_true_sorted = torch.sort(ys_true, descending=True)
    
    for i in range(len(ys_true_sorted[0]) - 1):
        for j in range(i, len(ys_true_sorted[0])):
            if (ys_pred[ys_true_sorted[1][i]] < ys_pred[ys_true_sorted[1][j]]) & \
                (ys_true_sorted[0][i] != ys_true_sorted[0][j]):
                n_pairs += 1
    
    return n_pairs

In [19]:
ys_true = torch.tensor([0., 2., 3., 3., 2.])
ys_pred = torch.tensor([0.2948, 0.0318, 0.0073, 0.4319, 0.7311])
num_swapped_pairs = 5

ys_true

tensor([0., 2., 3., 3., 2.])

In [24]:
torch.sort(ys_true, descending=True)

torch.return_types.sort(
values=tensor([3., 3., 2., 2., 0.]),
indices=tensor([2, 3, 1, 4, 0]))

In [31]:
ys_pred_sorted

torch.return_types.sort(
values=tensor([0.7311, 0.4319, 0.2948, 0.0318, 0.0073]),
indices=tensor([4, 3, 0, 1, 2]))

In [46]:
ys_pred_sorted[0].size()
ys_pred_sorted[0].shape

torch.Size([5])

In [47]:
len(ys_pred_sorted[0])

5

In [34]:
ys_pred_sorted[0][0]

tensor(0.7311)

In [29]:
ys_pred_sorted = torch.sort(ys_pred, descending=True)
ys_pred_sorted_idx = ys_pred_sorted[1]
ys_pred_sorted_idx

tensor([4, 3, 0, 1, 2])

In [30]:
ys_true_sorted = torch.sort(ys_true, descending=True)
ys_true_sorted_idx = ys_true_sorted[1]
ys_true_sorted_idx

tensor([2, 3, 1, 4, 0])

In [61]:
n_pairs = 0

for i in range(len(ys_pred_sorted[0]) - 1):
    for j in range(i, len(ys_pred_sorted[0])):
        if (ys_pred[ys_true_sorted[1][i]] < ys_pred[ys_true_sorted[1][j]]) & \
            (ys_true_sorted[0][i] != ys_true_sorted[0][j]):
            n_pairs += 1

n_pairs

5

# dcg ndcg with compute_gain

In [130]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'exp2':
        return 2 ** y_value - 1.
    else:
        return y_value + 0.

In [78]:
dir(sum(2 ** torch.tensor([2, 3])))

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__r

In [99]:
sum(2 ** torch.tensor([2, 3])).numpy()
sum(2 ** torch.tensor([2, 3])).item()

12

In [224]:
2 ** 3 - 1
tst3 = [math.log2(x) for x in range(2, 6)]
torch.tensor([2, 2, 3, 3]) / torch.tensor(tst3)
([2, 2, 3, 3]) # / tst3
tst3

[1.0, 1.584962500721156, 2.0, 2.321928094887362]

In [230]:
# def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
#     ys_pred_sorted = torch.sort(ys_pred, descending=True)
#     log2_tensor = torch.tensor([math.log2(x) for x in range(2, len(ys_pred) + 2)])
#     dcg_val = compute_gain(ys_true[ys_pred_sorted[1]], gain_scheme=gain_scheme) / \
#         log2_tensor
#     return sum(dcg_val).item()

def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    log2_list = [math.log2(x) for x in range(2, len(ys_pred) + 2)]
    
    dcg_val = 0.
    for i in range(len(log2_list)):
        dcg_val += compute_gain(ys_true[ys_pred_sorted[1]][i].item(), \
                                gain_scheme=gain_scheme) / log2_list[i]
    
    return dcg_val

In [231]:
tst2 = dcg(ys_true=torch.tensor([2, 3, 1, 4]), ys_pred = torch.tensor([4, 2, 1, 3]), gain_scheme='const')
tst2

6.454395572359223

In [214]:
ys_true = torch.tensor([2, 2, 2, 1, 1, 1, 3, 3, 3])
ys_pred = torch.tensor([1, 1, 1, 3, 3, 3, 2, 2, 2])
dcg(ys_true=ys_true, ys_pred=ys_pred, gain_scheme='const')

7.551795482635498

In [215]:
dcg(ys_true=ys_true, ys_pred=ys_pred, gain_scheme='exp2')

13.196569442749023

In [216]:
ys_true = torch.tensor([2, 3, 1, 4])
ys_pred = torch.tensor([4, 2, 1, 3])
ys_true

tensor([2, 3, 1, 4])

In [199]:
# math.log2(list(range(2, len(ys_true) + 1)))
# math.log2(ys_true)
torch.log2(ys_true + 0.)
torch.log2(torch.tensor(range(2, len(ys_true) + 2)).float())
# math.log2(torch.tensor(range(2, len(ys_true) + 2)).numpy())
# math.log2(torch.tensor(range(2, len(ys_true) + 2)).numpy().reshape(-1))
# math.log2(torch.tensor(range(2, len(ys_true) + 2)).numpy().flatten())
torch.tensor(range(2, len(ys_true) + 2)).numpy().reshape(-1)
torch.tensor(range(2, len(ys_true) + 2)).numpy().flatten()
torch.tensor(range(2, len(ys_true) + 2)).numpy()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [124]:
torch.tensor(range(3))

tensor([0, 1, 2])

In [129]:
ys_true / torch.log2(torch.tensor(range(2, len(ys_true) + 2)).float())

tensor([2.0000, 1.8928, 0.5000, 1.7227])

In [133]:
compute_gain(ys_true, gain_scheme='const') / \
    torch.log2(torch.tensor(range(2, len(ys_true) + 2)).float())

tensor([2.0000, 1.8928, 0.5000, 1.7227])

In [134]:
compute_gain(ys_true, gain_scheme='exp2') / \
    torch.log2(torch.tensor(range(2, len(ys_true) + 2)).float())

tensor([3.0000, 4.4165, 0.5000, 6.4601])

In [205]:
math.log2(3)

1.584962500721156

In [ ]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    ndcg_val = dcg(ys_true=ys_true, ys_pred=ys_pred, gain_scheme=gain_scheme) / \
        dcg(ys_true=ys_true, ys_pred=ys_true, gain_scheme=gain_scheme)
    return ndcg_val

# precision at k

In [ ]:
# def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
#     k = min(k, len(ys_true))
# #     ys_true_sorted = torch.sort(ys_true, descending=True)
#     ys_pred_sorted = torch.sort(ys_pred, descending=True)
#     is_pred_1 = [ys_pred_sorted[0].numpy() == 1]
#     is_pred_0 = [ys_pred_sorted[0].numpy() == 0]
#     print(ys_pred_sorted[0])
    
#     ys_true_sorted = ys_true[ys_pred_sorted[1]]
#     ys_true_sorted[is_pred_1] = torch.sort(ys_true_sorted[is_pred_1], descending=True)[0]
#     ys_true_sorted[is_pred_0] = torch.sort(ys_true_sorted[is_pred_0], descending=True)[0]
#     print(ys_true_sorted)
    
# #     ys_true_k = ys_true[ys_pred_sorted[1][:k]]
# #     ys_pred_k = ys_pred[ys_pred_sorted[1][:k]]
#     ys_true_k = ys_true_sorted[:k]
#     ys_pred_k = ys_pred_sorted[0][:k]
    
#     ys_pred_k_tp = sum((ys_true_k == ys_pred_k) & (ys_pred_k == 1)).item()
#     ys_true_k_tp = sum(ys_pred_k == 1).item()
    
#     if ys_true_k_tp == 0:
#         prec_k_val = -1
#     else:
#         prec_k_val = ys_pred_k_tp / ys_true_k_tp
    
#     return prec_k_val

In [117]:
# def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
#     k_max = k
#     k = min(k, len(ys_true))
#     ys_pred_sorted = torch.sort(ys_pred, descending=True)
    
# #     ys_true_sorted = torch.sort(ys_true, descending=True)
# #     is_pred_1 = [ys_pred_sorted[0].numpy() == 1]
# #     is_pred_0 = [ys_pred_sorted[0].numpy() == 0]
# #     print(ys_pred_sorted[0])
# #     ys_true_sorted = ys_true[ys_pred_sorted[1]]
# #     ys_true_sorted[is_pred_1] = torch.sort(ys_true_sorted[is_pred_1], descending=True)[0]
# #     ys_true_sorted[is_pred_0] = torch.sort(ys_true_sorted[is_pred_0], descending=True)[0]
# #     print(ys_true_sorted)

#     ys_true_sorted = ys_true[ys_pred_sorted[1]]
    
# #     if k:
# #         for i in range(1, len(ys_true)):
# #             y_pred_k_val = ys_pred_sorted[0][i]
# #             y_pred_k_idx = [ys_pred_sorted[0].numpy() == y_pred_k_val.item()]
# #             ys_true_sorted[y_pred_k_idx] = torch.sort(ys_true_sorted[y_pred_k_idx], descending=True)[0]
    
# #     ys_true_k = ys_true[ys_pred_sorted[1][:k]]
# #     ys_pred_k = ys_pred[ys_pred_sorted[1][:k]]
#     ys_true_k = ys_true_sorted[:k]
#     ys_pred_k = ys_pred_sorted[0][:k]
    
#     print(ys_true_k)
#     print(ys_pred_k)
    
# # #     ys_pred_k_tp = sum((ys_true_k == ys_pred_k) & (ys_pred_k == 1)).item()
# # #     ys_true_k_tp = sum(ys_pred_k == 1).item()
# # #     ys_true_k_tp = sum(ys_pred_k > 0).item()
#     ys_pred_k_tp = sum((ys_true_k == 1)).item()
#     ys_true_k_tp = len(ys_pred_k)
#     print(ys_pred_k_tp)
#     print(ys_true_k_tp)
    
#     if sum(ys_true_k).item() == 0:
#         prec_k_val = -1
#     else:
#         prec_max = 0
#         for i in range(1, len(ys_pred) + 1):
#             ys_pred_i_tp = sum((ys_true_sorted[:i] == 1)).item()
#             ys_true_i_tp = i
#             prec_max = max(prec_max, (ys_pred_i_tp / ys_true_i_tp))
            
#         prec_k_val = ys_pred_k_tp / ys_true_k_tp / prec_max
    
#     return prec_k_val

In [125]:
def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
    k_max = k
    k = min(k, len(ys_true))
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]]
    
#     if k:
#         for i in range(1, len(ys_true)):
#             y_pred_k_val = ys_pred_sorted[0][i]
#             y_pred_k_idx = [ys_pred_sorted[0].numpy() == y_pred_k_val.item()]
#             ys_true_sorted[y_pred_k_idx] = torch.sort(ys_true_sorted[y_pred_k_idx], descending=True)[0]
    
#     ys_true_k = ys_true[ys_pred_sorted[1][:k]]
#     ys_pred_k = ys_pred[ys_pred_sorted[1][:k]]
    ys_true_k = ys_true_sorted[:k]
    ys_pred_k = ys_pred_sorted[0][:k]
    
    print(ys_true_k)
    print(ys_pred_k)
    
# #     ys_pred_k_tp = sum((ys_true_k == ys_pred_k) & (ys_pred_k == 1)).item()
# #     ys_true_k_tp = sum(ys_pred_k == 1).item()
# #     ys_true_k_tp = sum(ys_pred_k > 0).item()

#     ys_pred_k_tp = sum((ys_true_k == 1)).item()
   
#     ys_true_k_tp = len(ys_pred_k)
# #     print(ys_pred_k_tp)
# #     print(ys_true_k_tp)
    
    if sum(ys_true).item() == 0:
        prec_k_val = -1
    else:
#         prec_max = 0
#         for i in range(1, len(ys_pred) + 1):
#             ys_pred_i_tp = sum((ys_true_sorted[:i] == 1)).item()
#             ys_true_i_tp = i
#             prec_max = max(prec_max, (ys_pred_i_tp / ys_true_i_tp))
#
#         prec_k_val = ys_pred_k_tp / ys_true_k_tp / prec_max
        
        prec_k_val = sum(ys_true_k).item() / min(sum(ys_true).item(), k)
    
    return prec_k_val

In [126]:
precission_at_k(\
    ys_true=torch.tensor([  0,   0,   1,    0,   1,   1,   0,   1,  1,   0,   0]), \
    ys_pred = torch.tensor([0.3, 0.9, 0.8, 0.8, 0.6, 0.5, 0.3, 0.3, 0.3, 0.2, 0.2]),\
    k = 8)

tensor([0, 1, 0, 1, 1, 0, 0, 1])
tensor([0.9000, 0.8000, 0.8000, 0.6000, 0.5000, 0.3000, 0.3000, 0.3000])


0.8

In [127]:
precission_at_k(\
    ys_true=torch.tensor([  0,   0,   1,    0,   1]), \
    ys_pred = torch.tensor([0.8, 0.8, 0.8, 0.8, 0.8]),\
    k = 5)

tensor([0, 0, 1, 0, 1])
tensor([0.8000, 0.8000, 0.8000, 0.8000, 0.8000])


1.0

In [319]:
ys_true = torch.tensor([2, 2, 2, 1, 1, 3, 1, 2, 3])
ys_pred = torch.tensor([3, 3, 3, 1, 1, 1, 2, 2, 2])
ys_true_sorted = torch.sort(ys_true, descending=True)
ys_true_sorted
ys_pred[[False,False,True,False,True,False,False,False,False]] = torch.tensor(77)
ys_pred
[ys_pred.numpy() == 77]
ys_true[[ys_pred.numpy() == 77]]
torch.sort(ys_pred, descending=True)

torch.return_types.sort(
values=tensor([77, 77,  3,  3,  2,  2,  2,  1,  1]),
indices=tensor([2, 4, 0, 1, 6, 7, 8, 3, 5]))

In [247]:
ys_true_sorted[1][:3]

tensor([6, 7, 8])

In [248]:
min(2,1)

1

In [249]:
len(ys_true)

9

In [262]:
(ys_true == ys_pred) & (ys_true == 1)

tensor(3)

In [257]:
(ys_true == 1)

tensor([False, False, False,  True,  True,  True, False, False, False])

In [260]:
sum((ys_true == 1)).item()

3

# reciprocal rank

In [166]:
def reciprocal_rank(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]]
    print(ys_pred_sorted)
    print(ys_true_sorted)
    
    if sum(ys_true_sorted.numpy() == 1) == 0:
        return 0
    else:
        ys_true_idx = ((ys_true_sorted == 1).nonzero(as_tuple=True)[0])
        return 1 / (ys_true_idx.item() + 1)
    

In [167]:
reciprocal_rank(\
    ys_true=torch.tensor([  0,   1,   0,    0,   0]), \
    ys_pred = torch.tensor([3, 2, 0.8, 5, 7]))

torch.return_types.sort(
values=tensor([7.0000, 5.0000, 3.0000, 2.0000, 0.8000]),
indices=tensor([4, 3, 0, 1, 2]))
tensor([0, 0, 0, 1, 0])
tensor([3])


0.25

In [155]:
ys_true = torch.tensor([  0,   0,   1,    0,   0])
ys_true == 1
ys_true.numpy() == 1

array([False, False,  True, False, False])

# p_found

In [170]:
def p_found(ys_true: torch.Tensor, ys_pred: torch.Tensor, p_break: float = 0.15) -> float:
    ys_pred_sorted = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_sorted[1]] # pRel
    
    pLook = torch.zeros(len(ys_true))
    pLook[0] = 1
    
    for i in range(1, len(ys_true)):
        pLook[i] = pLook[i - 1] * (1 - ys_true_sorted[i - 1]) * (1 - p_break)
    
    pFound = sum(pLook * ys_true_sorted).item()
    
    return pFound

In [171]:
p_found(\
    ys_true=torch.tensor([  0.5,   1,   0.7,    0.5,   0.3]), \
    ys_pred = torch.tensor([3, 2, 0.8, 5, 7]))

0.9550000429153442

# average precision

In [184]:
def average_precision(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    if sum(ys_true) == 0:
        return -1
    else:
        ys_pred_sorted = torch.sort(ys_pred, descending=True)
        ys_true_sorted = ys_true[ys_pred_sorted[1]]
        ys_true_sum = sum(ys_true).item()
        
        avg_prec = 0
        recall_k_prev = 0
        for i in range(len(ys_true)):
            recall_k = sum(ys_true_sorted[:(i + 1)]).item() / ys_true_sum
            prec_k = sum(ys_true_sorted[:(i + 1)]).item() / (i + 1)
            avg_prec_i = (recall_k - recall_k_prev) * prec_k
            recall_k_prev = recall_k
            
            avg_prec += avg_prec_i
            
        return avg_prec

In [185]:
average_precision(\
    ys_true=torch.tensor([  0.5,   1,   0.7,    0.5,   0.3]), \
    ys_pred = torch.tensor([3, 2, 0.8, 5, 7]))

0.5005555532375975